# KNN

## Variantes


- RadiusNeighborsClassifier: In cases where the data is not uniformly sampled. The user specifies a fixed radius r, such that points in sparser neighborhoods use fewer nearest neighbors for the classification. For high-dimensional parameter spaces, this method becomes less effective due to the so-called “curse of dimensionality”.
- KNeighborsClassifier: implements learning based on the k nearest neighbors of each query point, where k is an integer value specified by the user. In general, a larger k suppresses the effects of noise, but makes the classification boundaries less distinct.

# sklearn.neighbors.DistanceMetric


Metrics intended for integer-valued vector spaces: Though intended for integer-valued vectors, these are also valid metrics in the case of real-valued vectors.
identifier	class name	distance function
“hamming”	HammingDistance	N_unequal(x, y) / N_tot
No tan beneficioso: que si tenemos un dato donde aparece 10 veces la palabra 'viagra' y otro donde aparece 11 veces, no lo considera como cercano. (Por ahi tengamos que revisar los datos y separar en categorias: 'no aparece', 'aparece hasta 3 veces' y 'aparece mas de 3')
“canberra”	CanberraDistance	sum(|x - y| / (|x| + |y|))
“braycurtis”	BrayCurtisDistance	sum(|x - y|) / (sum(|x|) + sum(|y|))


En el caso que lleguemos a cambiar a los vectores por vectores booleanos: (no las mire, solo copie porque no creo que hagamos esto.)

Metrics intended for boolean-valued vector spaces: Any nonzero entry is evaluated to “True”. In the listings below, the following abbreviations are used:
N : number of dimensions
NTT : number of dims in which both values are True
NTF : number of dims in which the first value is True, second is False
NFT : number of dims in which the first value is False, second is True
NFF : number of dims in which both values are False
NNEQ : number of non-equal dimensions, NNEQ = NTF + NFT
NNZ : number of nonzero dimensions, NNZ = NTF + NFT + NTT
identifier	class name	distance function
“jaccard”	JaccardDistance	NNEQ / NNZ
“maching”	MatchingDistance	NNEQ / N
“dice”	DiceDistance	NNEQ / (NTT + NNZ)
“kulsinski”	KulsinskiDistance	(NNEQ + N - NTT) / (NNEQ + N)
“rogerstanimoto”	RogersTanimotoDistance	2 * NNEQ / (N + NNEQ)
“russellrao”	RussellRaoDistance	NNZ / N
“sokalmichener”	SokalMichenerDistance	2 * NNEQ / (N + NNEQ)
“sokalsneath”	SokalSneathDistance	NNEQ / (NNEQ + 0.5 * NTT)




Algo que se me ocurrio es que podemos hacer es mezclar un poco, separar el data frame entre los atributos booleanos y los que no lo son. Hacer KNN con ambos datos por separado y despues unir los resultados con algun peso para cada dato, pero no se. Lo vemos.

In [ ]:
# Esto agrega al python path el directorio ..
import sys
sys.path.append("..")

import json
import numpy as np
from dataframe_builder import DataFrameBuilder
# Ignorar Warning de sklearn (no es importante)
import warnings
warnings.filterwarnings('ignore',
                        message='Changing the shape of non-C contiguous array')


print "Leyendo archivos..."
ham, spam = json.load(open('../data/ham_dev.json')),json.load(open('../data/spam_dev.json'))

print "Construyendo datos"
builder = DataFrameBuilder()
df = builder.build(spam=spam, ham=ham)

print "Listo"

Leyendo archivos...
Construyendo datos


# Optimización de hiperparámetros

Busquemos los mejores (posibles) hiperparámetros

Para eso, primero veamos qué hiperparámetros nos provee la implementación de SKLearn.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

help(KNeighborsClassifier)

# En clase Agustin dijo :

- Técnica simple que a veces permite aproximar conceptos muy complejos.
- El entrenamiento es muy rápido. (¿Entrenamiento?)
- La consulta es muy lenta. (AyED eficientes.)
- El modelo (¿modelo?) ocupa mucho espacio en disco.
- Para pensar: La distancia se calcula con todos los atributos. ¿Qué pasa si algunos son irrelevantes?

Lo unico que se me ocurrio para responder es que se podra ahorrar bastante tiempo si no consideramos los atributos que estan al pepe. Para el informe estaria bueno volver a tirar knn despues de haber hecho pca o ica sobre los atributos y poner un timer a ver que onda.
Uds piensan que sera eso?

Opciones:

- n_neighbors: Cantidad de vecinos (?)
- weights: str
- splitter: No estoy muy seguro
- max_depth: máxima profundidad del árbol
- max_features: qué porcentaje de variables tomo a la hora de partir un nodo.
  Estas variables se eligen aleatoriamente
- min_samples_split: cuántos elementos tengo que tener en un nodo para decidir
  partirlo

In [ ]:
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.cross_validation import cross_val_score


options = {
    'n_neighbors': range(3, 100),
    'weights': ['uniform', 'distance'],
    'algorithm': ['ball_tree', 'kd_tree', 'brute'],
    # p es la p-distancia que usamos
    'p': range(1, 10)
    
}

# Preparo data para clasificar
X = df[builder.list_of_attributes].values
y = df['class'] == 'spam'

scoring_methods = ['precision', 'accuracy', 'f1', 'recall', 'roc_auc']

for scoring in scoring_methods:
    clf = KNeighborsClassifier()

    print("=" * 80 + "\n")
    print("Scoring {}".format(scoring))

    #search = GridSearchCV(clf, scoring=scoring, param_grid=options, n_jobs=4)
    search = RandomizedSearchCV(clf, scoring=scoring, param_distributions=options, n_jobs=4, n_iter=10)
    
    search.fit(X, y)
    
    print "Mejor combinación: {}".format(search.best_params_)
    print "Mejor valor: {}\n\n".format(search.best_score_)

    """
    Esto puede no tener mucho sentido. Estamos corriendo nuevamente cross_val_score... 
    """
    for other_scorer in scoring_methods:
        print "Valor para {} = {}".format(
            other_scorer,
            cross_val_score(search.best_estimator_, X, y, scoring=other_scorer, cv=10).mean()
        )
        
    

    